In [ ]:
import nltk
import math
import numpy as np
import pandas as pd
from nltk.corpus import treebank, stopwords

nltk.download('treebank', quiet=True)
nltk.download('stopwords', quiet=True)

stop_words = set(stopwords.words('english'))

file_ids = treebank.fileids()
documents = []
for file_id in file_ids:
    processed_words = [
        word.lower() for word in treebank.words(file_id) 
        if word.isalpha() and word.lower() not in stop_words
    ]
    documents.append(processed_words)

print(f"Корпус Treebank завантажено. Кількість документів: {len(documents)}")
print(f"Приклад першого обробленого документа (перші 15 слів): {documents[0][:15]}")

Корпус Treebank завантажено. Кількість документів: 199
Приклад першого обробленого документа (перші 15 слів): ['pierre', 'vinken', 'years', 'old', 'join', 'board', 'nonexecutive', 'director', 'vinken', 'chairman', 'elsevier', 'dutch', 'publishing', 'group']


# TASK 

In [ ]:
class TfidfVectorizer:
    def __init__(self):
        self.vocabulary = set()
        self.idf = {}
        self.vocab_map = {}

    def fit(self, documents):
        """
        Навчає модель: створює словник та обчислює IDF для кожного слова.
        """
        for doc in documents:
            self.vocabulary.update(doc)
        
        self.vocabulary = sorted(list(self.vocabulary))
        self.vocab_map = {word: i for i, word in enumerate(self.vocabulary)}
        
        num_documents = len(documents)
        df = {word: 0 for word in self.vocabulary}
        for word in self.vocabulary:
            for doc in documents:
                if word in doc:
                    df[word] += 1
        
        for word, doc_freq in df.items():
            self.idf[word] = math.log((num_documents + 1) / (doc_freq + 1))
            
    def transform(self, documents):
        """
        Перетворює документи на TF-IDF матрицю.
        """
        num_documents = len(documents)
        num_terms = len(self.vocabulary)
        tfidf_matrix = np.zeros((num_documents, num_terms))
        
        for doc_index, doc in enumerate(documents):
            term_counts = {word: doc.count(word) for word in set(doc)}
            
            for term, count in term_counts.items():
                if term in self.vocabulary:
                    term_index = self.vocab_map[term]
                    
                    tf = 1 + math.log(count)
                    
                    tfidf_score = tf * self.idf[term]
                    
                    tfidf_matrix[doc_index, term_index] = tfidf_score
                    
        return tfidf_matrix
    
    def fit_transform(self, documents):
        """
        Поєднує методи fit та transform.
        """
        self.fit(documents)
        return self.transform(documents)

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

print(f"\nРозмір отриманої TF-IDF матриці: {tfidf_matrix.shape}")
print("(рядки - документи, стовпці - унікальні слова)")

df_tfidf = pd.DataFrame(tfidf_matrix, columns=vectorizer.vocabulary)

print("\nTF-IDF матриця (перші 5 документів, перші 10 слів):")
print(df_tfidf.iloc[:5, :10].round(2))


Розмір отриманої TF-IDF матриці: (199, 8989)
(рядки - документи, стовпці - унікальні слова)

TF-IDF матриця (перші 5 документів, перші 10 слів):
   aba  abandon  abandoned  abating  abbey  abbot  abide  ability  able  \
0  0.0      0.0        0.0      0.0    0.0    0.0    0.0      0.0   0.0   
1  0.0      0.0        0.0      0.0    0.0    0.0    0.0      0.0   0.0   
2  0.0      0.0        0.0      0.0    0.0    0.0    0.0      0.0   0.0   
3  0.0      0.0        0.0      0.0    0.0    0.0    0.0      0.0   0.0   
4  0.0      0.0        0.0      0.0    0.0    0.0    0.0      0.0   0.0   

   aboard  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4     0.0  
